In [1]:
import jams
import muda
import numpy as np
import os
import gzip
import librosa
import medleydb as mdb
print('jams version: {:s}'.format(jams.__version__))
print('muda version: {:s}'.format(muda.__version__))
print('numpy version: {:s}'.format(np.__version__))
print('librosa version: {:s}'.format(librosa.__version__))
print('medleydb version: {:s}'.format(mdb.__version__))

jams version: 0.2.2
muda version: 0.1.2
numpy version: 1.12.0
librosa version: 0.5.0dev
medleydb version: 1.2.7


/home/js7561/dev/miniconda3/envs/py35hpcT9b/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Create JAMS files for the original data

In [2]:
def create_mdb_jam(mt):
    
    # Create jam
    jam = jams.JAMS()
    
    # Create annotation
    ann = jams.Annotation('tag_open')
    ann.duration = mt.duration
    
    # Add tag with snippet sound class
    ann.append(time=0, duration=mt.duration, value=mt.genre, confidence=1)
    
    # Fill file metadata
    jam.file_metadata.title = mt.title
    jam.file_metadata.release = '1.0'
    jam.file_metadata.duration = mt.duration
    jam.file_metadata.artist = mt.artist
    
    # Fill annotation metadata
    ann.annotation_metadata.version = '1.0'
    ann.annotation_metadata.corpus = 'MedleyDB'
    ann.annotation_metadata.data_source = 'http://medleydb.weebly.com/'
    
    # Store all metadata in sandbox too
    
    # Add annotation to jam
    jam.annotations.append(ann)
    
    # Return jam
    return jam

In [3]:
jamsfolder = '/scratch/js7561/datasets/MedleyDB_output/original/jams/'

In [4]:
mtracks = mdb.load_all_multitracks()

for n, mt in enumerate(mtracks):
    print(n, mt.track_id)
    jam = create_mdb_jam(mt)
    jamfile = os.path.join(jamsfolder, '{:s}.jams'.format(mt.track_id))
    jam.save(jamfile)

0 AClassicEducation_NightOwl
1 AimeeNorwich_Child
2 AimeeNorwich_Flying
3 AlexanderRoss_GoodbyeBolero
4 AlexanderRoss_VelvetCurtain
5 AmarLal_Rest
6 AmarLal_SpringDay1
7 Auctioneer_OurFutureFaces
8 AvaLuna_Waterduct
9 BigTroubles_Phantom
10 BrandonWebster_DontHearAThing
11 BrandonWebster_YesSirICanFly
12 CelestialShore_DieForUs
13 ChrisJacoby_BoothShotLincoln
14 ChrisJacoby_PigsFoot
15 ClaraBerryAndWooldog_AirTraffic
16 ClaraBerryAndWooldog_Boys
17 ClaraBerryAndWooldog_Stella
18 ClaraBerryAndWooldog_TheBadGuys
19 ClaraBerryAndWooldog_WaltzForMyVictims
20 Creepoid_OldTree
21 CroqueMadame_Oil
22 CroqueMadame_Pilot
23 Debussy_LenfantProdigue
24 DreamersOfTheGhetto_HeavyLove
25 EthanHein_1930sSynthAndUprightBass
26 EthanHein_BluesForNofi
27 EthanHein_GirlOnABridge
28 EthanHein_HarmonicaFigure
29 FacesOnFilm_WaitingForGa
30 FamilyBand_Again
31 Grants_PunchDrunk
32 Handel_TornamiAVagheggiar
33 HeladoNegro_MitadDelMundo
34 HezekiahJones_BorrowedHeart
35 HopAlong_SisterCities
36 InvisibleFamil

## Setup deformers

In [4]:
# PITCH1
pitch_shifts1 = [-2, -1, 1, 2]
pitch1 = muda.deformers.PitchShift()

# PITCH2
pitch_shifts2 = [-4, -3, 3, 4]
pitch2 = muda.deformers.PitchShift()

# DRC (compression)
drc_presets1 = ['radio', 'film standard', 'music standard', 'speech']
drc1 = muda.deformers.DynamicRangeCompression(preset=drc_presets1)

# TIME STRETCH
stretch1 = muda.deformers.LogspaceTimeStretch(n_samples=4, lower=-0.3, upper=0.3)

deformers = {'pitch1': pitch1, 'pitch2': pitch2, 'drc1': drc1, 'stretch1': stretch1}
deformer_params = {'pitch1': pitch_shifts1, 'pitch2': pitch_shifts2, 'drc1': None, 'stretch1': None}

## Deformation utility function

In [7]:
# pitch deformation function
def apply_deformer(track_id, audiofile, jamsfile, outputfolder, deformer=None, pitch_shifts=None, 
                   dname=None, sr=44100):
    
    # Create folders
    deformerfolder = os.path.join(outputfolder, dname)
    if not os.path.isdir(deformerfolder):
        os.mkdir(deformerfolder)
    
    audiofolder = os.path.join(deformerfolder, 'audio')
    if not os.path.isdir(audiofolder):
        os.mkdir(audiofolder)

    jamsfolder = os.path.join(deformerfolder, 'jams')
    if not os.path.isdir(jamsfolder):
        os.mkdir(jamsfolder)
        
    jorig = muda.load_jam_audio(jamsfile, audiofile, sr=sr)

    # Perform deformations
    if pitch_shifts is not None:
        for n, shift in enumerate(pitch_shifts):
            deformer.n_semitones = shift
            for j in deformer.transform(jorig):
                aug_audiofile = os.path.join(audiofolder, '{:s}_{:s}_{:d}.wav'.format(track_id, dname, n))
                aug_jamsfile = os.path.join(jamsfolder, '{:s}_{:s}_{:d}.jams'.format(track_id, dname, n))
                muda.save(aug_audiofile, aug_jamsfile, j)
    else:        
        # All other deformers
        for n, j in enumerate(deformer.transform(jorig)):
            aug_audiofile = os.path.join(audiofolder, '{:s}_{:s}_{:d}.wav'.format(track_id, dname, n))
            aug_jamsfile = os.path.join(jamsfolder, '{:s}_{:s}_{:d}.jams'.format(track_id, dname, n))
            muda.save(aug_audiofile, aug_jamsfile, j)

## Perform all deformations

In [ ]:
outputfolder = '/scratch/js7561/datasets/MedleyDB_output/'
sr = 44100

mtracks = mdb.load_all_multitracks()

for n, mt in enumerate(mtracks):
    print(n, mt.track_id)
    jamfile = os.path.join(jamsfolder, '{:s}.jams'.format(mt.track_id))
    audiofile = mt.mix_path
    
    for dname in deformers.keys():
        apply_deformer(mt.track_id, audiofile, jamfile, outputfolder, 
                       deformer=deformers[dname], pitch_shifts=deformer_params[dname], 
                       dname=dname, sr=sr)

0 AClassicEducation_NightOwl


/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <clas

1 AimeeNorwich_Child


/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <clas

2 AimeeNorwich_Flying


...

/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <clas

20 Creepoid_OldTree


/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <clas

21 CroqueMadame_Oil


/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
/home/js7561/dev/librosa/librosa/core/pitch.py:157: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  bins = np.linspace(-0.5, 0.5, np.ceil(1./resolution), endpoint=False)
